In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pymc3 as pm
from pymc3 import Model, Normal, Slice
from pymc3 import sample
from pymc3 import traceplot
from pymc3.distributions import Interpolated
from theano import as_op
import theano.tensor as tt
import numpy as np
from scipy import stats
import theano

plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(pm.__version__))

Running on PyMC3 v3.5


In [20]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha_true = 5
beta0_true = 7
beta1_true = 13
beta2_true = 10
# Size of dataset
size = 100

# Predictor variable
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2
X3 = np.random.randn(size) * 0.3

# Simulate outcome variable
Y = alpha_true + beta0_true * X1 + beta1_true * X2 + beta2_true * X3 + np.random.randn(size)

In [22]:
x_all = np.array([X1, X2, X3])
x = theano.shared(x_all)


In [24]:
basic_model = Model()

with basic_model:

    # Priors for unknown model parameters
    alpha = Normal('alpha', mu=0, sd=1)
    beta0 = Normal('beta0', mu=12, sd=1)
    beta1 = Normal('beta1', mu=18, sd=1)
    beta2 = Normal('beta2', mu=15, sd=1)

    # Expected value of outcome
    mu = alpha + beta0 * x_all[0] + beta1 * x_all[1] + beta2*x_all[2]

    # Likelihood (sampling distribution) of observations
    Y_obs = Normal('Y_obs', mu=mu, sd=1, observed=Y)

    # draw 1000 posterior samples
    trace = sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta2, beta1, beta0, alpha]
Sampling 4 chains: 100%|██████████| 6000/6000 [00:02<00:00, 2931.73draws/s]


In [15]:
# pm.traceplot(trace);

In [25]:
def from_posterior(param, samples):
    smin, smax = np.min(samples), np.max(samples)
    width = smax - smin
    x = np.linspace(smin, smax, 100)
    y = stats.gaussian_kde(samples)(x)

    # what was never sampled should have a small probability but not 0,
    # so we'll extend the domain and use linear approximation of density on it
    x = np.concatenate([[x[0] - 3 * width], x, [x[-1] + 3 * width]])
    y = np.concatenate([[0], y, [0]])
    
    return Interpolated(param, x, y)

In [26]:
traces = [trace]

In [27]:
for _ in range(10):

    # generate more data
    X1 = np.random.randn(size)
    X2 = np.random.randn(size) * 0.2
    X3 = np.random.randn(size) * 0.3
    Y = alpha_true + beta0_true * X1 + beta1_true * X2 + beta2_true * X3 + np.random.randn(size)
    
    x_temp = np.array([X1,X2,X3])
    x.set_value(x_temp)
    
    model = Model()
    with model:
        # Priors are posteriors from previous iteration
        alpha = from_posterior('alpha', trace['alpha'])
        beta0 = from_posterior('beta0', trace['beta0'])
        beta1 = from_posterior('beta1', trace['beta1'])
        beta2 = from_posterior('beta2', trace['beta2'])
        # Expected value of outcome
        mu = alpha + beta0 * x[0] + beta1 * x[1] + beta2 * x[2]

        # Likelihood (sampling distribution) of observations
        Y_obs = Normal('Y_obs', mu=mu, sd=1, observed=Y)

        # draw 10000 posterior samples
        trace = sample(1000)
        traces.append(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta2, beta1, beta0, alpha]
Sampling 4 chains: 100%|██████████| 6000/6000 [00:03<00:00, 1623.02draws/s]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta2, beta1, beta0, alpha]
Sampling 4 chains: 100%|██████████| 6000/6000 [00:03<00:00, 1531.26draws/s]
The number of effective samples is smaller than 25% for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta2, beta1, beta0, alpha]
Sampling 4 chains: 100%|██████████| 6000/6000 [00:03<00:00, 1617.53draws/s]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta2, beta1, beta0, alpha]
Sampling 4 chains: 100%|██████████| 6000/6000 [00:03<00:00, 1581.78draws/s]
Auto-as

KeyboardInterrupt: 

In [30]:
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2
x_temp = np.array([X1,X2])
x.set_value(x_temp)

In [31]:
ppc = pm.sample_ppc(trace, samples=50, model=basic_model)
y_obs = np.array(ppc['Y_obs'])

100%|██████████| 50/50 [00:00<00:00, 93.06it/s]
